In [1]:
# train_model.py
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import joblib
import tensorflow as tf

# 1. Veriyi Yükle
df = pd.read_csv("stocks.csv")

# Sadece GOOG hissesini seçelim (Eğitim için odaklanmak en iyisidir)
data = df[df['Ticker'] == 'GOOG'].copy()

# Tarihe göre sırala
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

# 2. Kullanılacak Özellikler (Multivariate)
# Girdiler: Open, High, Low, Volume, Close
# Hedef: Close (Gelecekteki kapanış fiyatı)
features = ['Open', 'High', 'Low', 'Volume', 'Close']
dataset = data[features].values

# 3. Ölçeklendirme (MinMax Scaling - LSTM için çok önemlidir)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# 4. Zaman Serisi Dizileri Oluşturma (Sliding Window)
def create_sequences(data, seq_length=60):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i]) # Geçmiş 60 gün (Tüm özellikler)
        y.append(data[i, 4]) # 4. indeks 'Close' sütunudur (Hedef)
    return np.array(X), np.array(y)

SEQ_LENGTH = 60
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Veriyi Eğitim/Test olarak ayırma (%80 Eğitim)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 5. LSTM Model Mimarisi
print("LSTM Modeli kuruluyor...")
model = Sequential()

# 1. LSTM Katmanı
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# 2. LSTM Katmanı
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Çıktı Katmanı
model.add(Dense(units=1)) # Tek bir değer tahmin ediyoruz (Close Price)

model.compile(optimizer='adam', loss='mean_squared_error')

# 6. Modeli Eğitme
print("Eğitim başlıyor...")
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# 7. Kaydetme
model.save('stock_lstm_model.h5')
joblib.dump(scaler, 'stock_scaler.pkl')
print("Model ve Scaler kaydedildi.")

C:\Users\wfi-7\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


LSTM Modeli kuruluyor...
Eğitim başlıyor...
Epoch 1/20


C:\Users\wfi-7\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 1.0862 - val_loss: 0.6868
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.2744 - val_loss: 0.4540
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 1.0109 - val_loss: 0.2627
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.1017 - val_loss: 0.1216
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.1298 - val_loss: 0.0333
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.0055 - val_loss: 7.7418e-04
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 7.8734e-05 - val_loss: 0.0125
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0352 - val_loss: 0.0397
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0857 - val_loss: 0.0528
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0088 - val_loss: 0.0601
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.2066 - val_loss: 0.0455
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.0595 - val_loss: 0.0270
Epoch

Model ve Scaler kaydedildi.
